In [ ]:
from operetta_compose.tasks.harmony_to_ome_zarr import _parse_harmony_index
from pathlib import Path


well_df, img_df = _parse_harmony_index(Path("tests/operetta_plate/Images/"))

In [ ]:
img_df

In [ ]:
img_df.groupby(["row", "col"]).agg(list)

In [ ]:

import numpy as np
import tifffile


class TiffLoader:
    """Png loader for tiled images."""

    def __init__(self, dir: str, paths: list[str]):
        """Initialize the PngLoader.

        Args:
            dir (str): Directory where the PNG files are located.
            paths (list[str]): List of PNG file paths relative to the directory.
        """
        self.dir = dir
        self.paths = paths

    def load(self) -> np.ndarray:
        """Load the PNG files and stack them into a numpy array."""
        arrays = []
        for path in self.paths:
            full_path = f"{self.dir}/{path}"
            img = tifffile.imread(full_path)
            arrays.append(np.array(img))
        stacked_arrays = np.stack(arrays, axis=0)[None, None, :, :, :]
        return stacked_arrays

    @property
    def dtype(self):
        """Return the data type of the loaded images."""
        return "uint8"

In [ ]:
from fractal_converters_tools import PlatePathBuilder, TiledImage
import numpy as np
from ngio import PixelSize

from fractal_converters_tools import OriginDict, Point, Tile, Vector

alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

tiled_images = []
for image in img_df.groupby(["row", "col"]).agg(list).itertuples():
    tiled_image = TiledImage(
        name="None",
        path_builder=PlatePathBuilder(
                plate_name="operetta_plate",
                row=alphabet[image.Index[0] - 1],
                column=image.Index[1],
                acquisition_id=0,  
            ),
        channel_names=[image.ChannelName[0]],
    )
    
    # filter the img_df to only have row == image.Index[0] and col == image.Index[1]
    img_subset = img_df[(img_df["row"] == image.Index[0]) & (img_df["col"] == image.Index[1])]
    for tile_df in img_subset.itertuples():
        # 1) Define pixel size
        pixel_size = PixelSize(
            x=tile_df.res_x,
            y= tile_df.res_y,
            z=tile_df.res_z,
            t=1
        )
        
        # 2) Define the top left point of the tile
        top_left = Point(
                x=tile_df.pos_x,
                y=tile_df.pos_y,
                z=0, # to be refined with the z position
                t=0,  # to be refined with the t position
                c=tile_df.channel - 1, # channel is 1-indexed in the dataframe, but 0-indexed in the Point object
            )
        
        # 3) Define the tile diagonal "Vector"
        # NB in the previous version of the code, I was using the bottom right point because I was following the example 
        # bug in the operetta metadata, but it seems that diagon is actually easier to use.
        # Also using the diagonal Vector in this case avoid the one-off error that we observed in the previous version
        diag = Vector(
            x=tile_df.len_x,
            y=tile_df.len_y,
            z=tile_df.len_z,
            c=tile_df.channel, # This should match the number of channels in the image
            t=1, # to be refined with the end t position/number of time points
        )
        
        loader = TiffLoader(
            dir="tests/operetta_plate/Images/",
            paths=[tile_df.img_name]
            )
        
        tile_obj = Tile(
            top_l=top_left,
            diag=diag,
            pixel_size=pixel_size,
            origin=None, # Origin can be set to None if is the same as the top left point
            shape=(1, 1, 1, 1080, 1080),  # This shape parame is optional and can be inferred from the diag vector, the reason to keep it is to avoid potential one-off errors
            data_loader=loader,
        )
        tiled_image.add_tile(tile_obj)
    tiled_images.append(tiled_image)

In [ ]:
from fractal_converters_tools import (
    AdvancedComputeOptions,
    build_parallelization_list,
    initiate_ome_zarr_plates,
)

# Build parallelization list for the compute phase
parallelization_list = build_parallelization_list(
    zarr_dir="./test_zarr",
    tiled_images=tiled_images,
    overwrite=True,
    advanced_compute_options=AdvancedComputeOptions(),
)

# Initiate the OME-Zarr plates with the tiled image
initiate_ome_zarr_plates(
    zarr_dir="./test_zarr",
    tiled_images=tiled_images,
    overwrite=True,
)

In [ ]:
from fractal_converters_tools import ConvertParallelInitArgs, generic_compute_task

for task in parallelization_list:
    print(f"Running task: {task}")
    generic_compute_task(
        zarr_url=task["zarr_url"],
        init_args=ConvertParallelInitArgs(**task["init_args"]),
    )